# Práctica sobre algoritmos genéticos

Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.

Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


In [28]:
import random
import numpy
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

## Trenes

In [ ]:
class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)


In [30]:
def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list

trains = random_trains_generation(100)

In [31]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [32]:
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual, trains)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [33]:
def evalTrains(individual):
    use_times = {"op1": 0, "op2": 0, "op3": 0}
    total_time = 0

    for train in individual:
        if use_times[train.op] == 0:
            use_times[train.op] = train.wagons
        else:
            waiting_time = use_times[train.op]
            for op, value in use_times.items():
                if value - waiting_time < 0:
                    use_times[op] = 0
                else:
                    use_times[op] = value - waiting_time
            use_times[train.op] = train.wagons
            total_time += waiting_time

    total_time += max(use_times.values())
    return total_time,

In [34]:
licence_plates = set()
for t in trains:
    licence_plates.add(t.licence_plate)

In [35]:
def get_missing_trains(ind):
    missing_ind = licence_plates - set(train.licence_plate for train in ind)
    return [t for t in trains if t.licence_plate in missing_ind]

def remove_repeated(ind):
    return [t for i, t in enumerate(ind) if t.licence_plate not in {x.licence_plate for x in ind[:i]}]

def cxOnePointModified(ind1, ind2):
    cxpoint = random.randint(0,len(ind1) - 1)

    # Intercambiar los segmentos seleccionados entre los padres
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    missing_ind1 = get_missing_trains(ind1)
    missing_ind2 = get_missing_trains(ind2)
    none_repeated_ind1 = remove_repeated(ind1)
    none_repeated_ind2 = remove_repeated(ind2)
    
    ind1[:] = none_repeated_ind1 + missing_ind1
    
    ind2[:] = none_repeated_ind2 + missing_ind2

    return ind1, ind2

In [36]:
toolbox.register("evaluate", evalTrains)
toolbox.register("mate", cxOnePointModified)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=4)

In [37]:
def main():
    NGEN = 500
    MU = 10
    CXPB = 0.7
    MUTPB = 0.2

    # Población inicial
    pop = toolbox.population(n=MU)
    
    # Estadísticas de la población
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    # Hall Of Fame
    hof = tools.HallOfFame(4)


    algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

In [38]:
pop, stats, hof  = main()

gen	nevals	avg    	min    	max    
0  	10    	[1085.]	[1085.]	[1085.]
1  	10    	[1084.6]	[1054.]	[1112.]
2  	10    	[1066.5]	[1012.]	[1092.]
3  	6     	[1045.3]	[1012.]	[1087.]
4  	8     	[1022.] 	[999.] 	[1041.]
5  	6     	[1012.8]	[962.] 	[1073.]
6  	6     	[998.]  	[962.] 	[1037.]
7  	8     	[982.7] 	[962.] 	[999.] 
8  	0     	[970.2] 	[962.] 	[979.] 
9  	10    	[964.4] 	[962.] 	[970.] 
10 	5     	[963.3] 	[962.] 	[975.] 
11 	9     	[966.7] 	[962.] 	[1005.]
12 	5     	[964.]  	[962.] 	[982.] 
13 	9     	[971.8] 	[962.] 	[1046.]
14 	8     	[963.7] 	[962.] 	[972.] 
15 	8     	[967.1] 	[922.] 	[1007.]
16 	5     	[945.7] 	[922.] 	[962.] 
17 	9     	[945.6] 	[918.] 	[1019.]
18 	6     	[927.1] 	[899.] 	[982.] 
19 	7     	[918.6] 	[882.] 	[955.] 
20 	5     	[903.5] 	[882.] 	[948.] 
21 	6     	[884.9] 	[871.] 	[908.] 
22 	9     	[897.3] 	[867.] 	[990.] 
23 	8     	[870.3] 	[856.] 	[889.] 
24 	5     	[873.]  	[836.] 	[930.] 
25 	8     	[857.4] 	[831.] 	[935.] 
26 	6     	[843.8] 	[831.] 	[8

### Tiempo Ideal vs Tiempo del algoritmo

In [39]:
time_of_operations = {"op1": 0, "op2": 0, "op3": 0}
for t in trains:
    time_of_operations[t.op] += t.wagons
print("el tiempo mínimo será: ",max(time_of_operations["op1"], time_of_operations["op2"], time_of_operations["op3"]))
print("el tiempo del algoritmo será: ",hof[0].fitness.values[0])

el tiempo mínimo será:  719
el tiempo del algoritmo será:  752.0


### Revisión de resultados repetidos

In [27]:
licence_plates = set()
licencias = []
for t in hof[0]:
    licence_plates.add(t.licence_plate)
    licencias.append(t.licence_plate)

print(len(hof[0]))
print(licence_plates)
print(licencias)

100
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}
[30, 71, 2, 25, 4, 12, 47, 83, 8, 9, 10, 11, 76, 45, 14, 60, 64, 18, 26, 31, 19, 37, 84, 46, 51, 93, 87, 77, 21, 33, 54, 32, 13, 34, 15, 24, 85, 38, 39, 27, 72, 16, 42, 44, 81, 35, 50, 22, 63, 91, 88, 5, 57, 59, 28, 61, 1, 20, 65, 43, 67, 29, 69, 70, 41, 73, 53, 75, 0, 96, 36, 80, 66, 82, 7, 23, 74, 86, 78, 58, 90, 92, 68, 94, 95, 40, 97, 98, 99, 17, 3, 6, 48, 49, 55, 79, 62, 56, 89, 52]
